In [1]:
using CSV
using DataFrames
using Statistics
# Ruta al archivo CSV
file_path = "Datos_Práctica_Evaluación_1.csv"

# Cargar los datos
data = CSV.read(file_path, DataFrame)

# Descripción de los datos
println("Descripción de los datos:")
println("Número de variables (columnas): ", size(data, 2)-2)
println("Número de instancias (filas): ", size(data, 1))
println("Número de individuos únicos: ", length(unique(data.subject)))
println("Número de clases de salida: ", length(unique(data.Activity)))
missing_percentage_per_column = map(col -> mean(ismissing, col) * 100, eachcol(data))
println("Porcentaje de nulos por variable:", missing_percentage_per_column)
println("Media de todos los porcentajes de nulo: ",sum(missing_percentage_per_column) / length(missing_percentage_per_column))


Descripción de los datos:
Número de variables (columnas): 561
Número de instancias (filas): 10299
Número de individuos únicos: 30
Número de clases de salida: 6
Porcentaje de nulos por variable:[0.0, 0.0, 0.0, 0.0, 0.01941936110301971, 0.0, 0.01941936110301971, 0.01941936110301971, 0.029129041654529564, 0.029129041654529564, 0.01941936110301971, 0.0, 0.009709680551509855, 0.0, 0.0, 0.0, 0.009709680551509855, 0.0, 0.0, 0.0, 0.029129041654529564, 0.009709680551509855, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.009709680551509855, 0.0, 0.0, 0.03883872220603942, 0.03883872220603942, 0.01941936110301971, 0.009709680551509855, 0.0, 0.0, 0.0, 0.0, 0.0, 0.029129041654529564, 0.009709680551509855, 0.009709680551509855, 0.0, 0.0, 0.

In [2]:
using CSV
using DataFrames
using Statistics
using MLJ  # Para técnicas de preprocesamiento

# Cargar datos
file_path = "C:\\Users\\leonf\\OneDrive\\Documentos\\AA_PRIMER_CUATRI\\MAAAI\\MAAA1\\Datos_Práctica_Evaluación_1.csv"
data = CSV.read(file_path, DataFrame)

# Rellenar valores faltantes
for col in names(data)
    if eltype(data[!, col]) <: Union{Missing, Number}  # Para columnas numéricas
        mean_value = mean(skipmissing(data[!, col]))  # Promedio de valores existentes
        if eltype(data[!, col]) <: Int  # Si la columna es de tipo entero
            mean_value = Int(round(mean_value))  # Redondear y convertir a entero
        end
        data[ismissing.(data[!, col]), col] .= mean_value  # Rellenar valores faltantes
    elseif eltype(data[!, col]) <: Union{Missing, AbstractString}  # Para columnas de texto
        mode_value = mode(skipmissing(data[!, col]))  # Valor más frecuente
        data[ismissing.(data[!, col]), col] .= mode_value
    end
end

println("Relleno de valores faltantes completado.")


# 3. Codificar variables categóricas
categorical_cols = filter(col -> eltype(data[!, col]) <: AbstractString, names(data))
println("Variables categóricas detectadas: ", categorical_cols)
for col in categorical_cols
    data[!, col] = categorical(data[!, col])  # Convertir a tipo categórico
end

# 4. Escalar variables numéricas
numerical_cols = filter(col -> eltype(data[!, col]) <: Number, names(data))
for col in numerical_cols
    col_values = data[!, col]
    data[!, col] = (col_values .- mean(col_values)) ./ std(col_values)  # Z-score
end

println("Transformación de datos completada.")

# 5. Verificar los datos transformados
println("Primeras filas de los datos:")
println(first(data, 5))


Relleno de valores faltantes completado.
Variables categóricas detectadas: ["Activity"]
Transformación de datos completada.
Primeras filas de los datos:
5×563 DataFrame
 Row │ subject   tBodyAcc-mean()-X  tBodyAcc-mean()-Y  tBodyAcc-mean()-Z  tBodyAcc-std()-X  tBodyAcc-std()-Y  tBodyAcc-std()-Z  tBodyAcc-mad()-X  tBodyAcc-mad()-Y  tBodyAcc-mad()-Z  tBodyAcc-max()-X  tBodyAcc-max()-Y  tBodyAcc-max()-Z  tBodyAcc-min()-X  tBodyAcc-min()-Y  tBodyAcc-min()-Z  tBodyAcc-sma()  tBodyAcc-energy()-X  tBodyAcc-energy()-Y  tBodyAcc-energy()-Z  tBodyAcc-iqr()-X  tBodyAcc-iqr()-Y  tBodyAcc-iqr()-Z  tBodyAcc-entropy()-X  tBodyAcc-entropy()-Y  tBodyAcc-entropy()-Z  tBodyAcc-arCoeff()-X,1  tBodyAcc-arCoeff()-X,2  tBodyAcc-arCoeff()-X,3  tBodyAcc-arCoeff()-X,4  tBodyAcc-arCoeff()-Y,1  tBodyAcc-arCoeff()-Y,2  tBodyAcc-arCoeff()-Y,3  tBodyAcc-arCoeff()-Y,4  tBodyAcc-arCoeff()-Z,1  tBodyAcc-arCoeff()-Z,2  tBodyAcc-arCoeff()-Z,3  tBodyAcc-arCoeff()-Z,4  tBodyAcc-correlation()-X,Y  tBodyAcc-correlation()-X,Z

In [7]:
using Random
using DataFrames
using StatsBase  

# Semilla para reproducibilidad
seed = 172
Random.seed!(seed)

# Supongamos que los datos están cargados en un DataFrame llamado `data`
# Y que la columna `subject` identifica a cada individuo.

# Obtener la lista única de individuos
individuos = unique(data.subject)

# Seleccionar el 10% de los individuos al azar
num_holdout = ceil(Int, length(individuos) * 0.1)  # Redondeo hacia arriba para asegurar al menos 1
holdout_individuos = sample(individuos, num_holdout, replace=false)
# Dividir los datos en train y holdout
holdout_data = filter(row -> row.subject in holdout_individuos, data)
train_data = filter(row -> !(row.subject in holdout_individuos), data)

# Mostrar los individuos seleccionados para el holdout
println("Individuos en el conjunto de holdout: ", holdout_individuos)

# Mostrar resumen de los conjuntos resultantes
println("Número de filas en el conjunto de entrenamiento: ", nrow(train_data))
println("Número de filas en el conjunto de holdout: ", nrow(holdout_data))


Individuos en el conjunto de holdout: [-1.6299473741119903, -0.36252999202505987, -0.1320904680092543]
Número de filas en el conjunto de entrenamiento: 9342
Número de filas en el conjunto de holdout: 957


In [17]:
using Random
using DataFrames
using MLJ

# Asegurar reproducibilidad
Random.seed!(172)

# Asumimos que 'train_data' es el DataFrame que contiene los datos de entrenamiento
# después de haber separado el holdout

# Obtener la lista única de participantes en el conjunto de entrenamiento
participantes = unique(train_data.subject)

# Crear un vector para almacenar los índices de los folds
folds = Vector{Vector{Int}}(undef, 5)

# Dividir los participantes en 5 grupos
participantes_shuffled = shuffle(participantes)
grupos_participantes = [participantes_shuffled[i:5:end] for i in 1:5]

# Asignar las instancias a los folds basándose en los grupos de participantes
for (i, grupo) in enumerate(grupos_participantes)
    folds[i] = findall(x -> x in grupo, train_data.subject)
end

# Verificar que cada instancia está en exactamente un fold
total_instancias = sum(length.(folds))
@assert total_instancias == nrow(train_data) "Error en la división de folds"

# Imprimir información sobre los folds
for (i, fold) in enumerate(folds)
    println("Fold $i: $(length(fold)) instancias, $(length(unique(train_data.subject[fold]))) participantes")
end

# Función para realizar el entrenamiento y evaluación en cada fold
function evaluar_fold(modelo, X, y, train_indices, test_indices)
    # Entrenar el modelo
    mach = machine(modelo, X, y)
    fit!(mach, rows=train_indices)
    
    # Evaluar el modelo
    y_pred = predict(mach, rows=test_indices)
    
    # Calcular y retornar la métrica de evaluación (por ejemplo, accuracy)
    return accuracy(y[test_indices], y_pred)
end

function comprobar_individuos_unicos(folds, data)
    for i in 1:length(folds)
        individuos_fold_i = Set(data.subject[folds[i]])
        for j in (i+1):length(folds)
            individuos_fold_j = Set(data.subject[folds[j]])
            if !isempty(intersect(individuos_fold_i, individuos_fold_j))
                println("Error: Los folds $i y $j comparten individuos.")
                return false
            end
        end
    end
    println("Verificación exitosa: No hay individuos compartidos entre los folds.")
    return true
end
# Después de crear tus folds
if comprobar_individuos_unicos(folds, train_data)
    println("La división en folds es correcta.")
else
    println("Hay un problema con la división en folds.")
end



Fold 1: 2170 instancias, 6 participantes
Fold 2: 2132 instancias, 6 participantes
Fold 3: 1688 instancias, 5 participantes
Fold 4: 1733 instancias, 5 participantes
Fold 5: 1619 instancias, 5 participantes
Verificación exitosa: No hay individuos compartidos entre los folds.
La división en folds es correcta.


In [18]:
# Contar participantes originales
total_participantes_original = length(unique(data.subject))

# Contar participantes en el conjunto de entrenamiento
participantes_train = length(unique(train_data.subject))

# Contar participantes en el holdout
participantes_holdout = length(unique(holdout_data.subject))

println("Participantes originales: ", total_participantes_original)
println("Participantes en train: ", participantes_train)
println("Participantes en holdout: ", participantes_holdout)


Participantes originales: 30
Participantes en train: 27
Participantes en holdout: 3
